In [1]:
%cd /ibex/user/slimhy/PADS/code/
"""
Extracting features into HDF5 files for each split.
"""
import argparse
import json
import os
import torch
import trimesh
import zipfile

import util.misc as misc
import util.s2vs as s2vs

from datasets.CoMPaT.compat3D import SegmentedMeshLoader
from datasets.CoMPaT.utils3D.plot import label_to_RGB, FINE_RGB_RANGE
from datasets.metadata import (
    COMPAT_CLASSES,
    int_to_hex,
)
from util.misc import CUDAMesh, d_GPU, show_side_by_side
from voxelize.preprocess import mesh_to_manifold_plus

/ibex/user/slimhy/PADS/code


In [2]:
ZIP_PATH = "/ibex/project/c2273/3DCoMPaT/3DCoMPaT_ZIP.zip"
META_DIR = "/ibex/project/c2273/3DCoMPaT/3DCoMPaT-v2/metadata"
OUT_PATH = "/ibex/project/c2273/3DCoMPaT/manifold"

In [6]:
# Instantiating segment dataset
seg_dataset = SegmentedMeshLoader(
    zip_path=ZIP_PATH,
    meta_dir=META_DIR,
    split="all",
    shuffle=True,
    seed=0,
)

N_ITEMS = range(len(seg_dataset))
for mesh_id in N_ITEMS:
    model_id = seg_dataset.shape_ids[mesh_id]

    # Get the mesh labels
    manifold_map = {}
    for mesh_label, mesh in seg_dataset[mesh_id].items():
        if not mesh.is_watertight:
            mesh = mesh_to_manifold_plus(mesh, depth=8)
        manifold_map[mesh_label] = mesh
        
    # Show one of the parts
    manifold_part = list(manifold_map.values())[3]
    manifold_part.show()
    
    # Make the merged mesh watertight
    all_meshes = trimesh.util.concatenate([mesh for mesh in manifold_map.values()])
    if not all_meshes.is_watertight:
        manifold_mesh = mesh_to_manifold_plus(all_meshes, depth=8)

    # Pickle manifold map and dump to output directory
    misc.dump_pickle(manifold_map, os.path.join(OUT_PATH, f"{model_id}.pkl"))

    # Export the watertight mesh
    manifold_mesh.export(os.path.join(OUT_PATH, f"{model_id}.obj"))
    
    break

##################### Arguments #####################
depth: 8.
input: /tmp/tmp_817100.obj.
output: /tmp/tmpout_817100.obj.
#####################################################
vertex number: 6353    face number: 3152
Initialize AABB Tree...
Build Halfedges...
Split non-manifold vertices...
Rebuild Halfedges...
Gauss-seidel update...
Iter 71 with active vertex number 55      
Sharp preservation...
Gauss-seidel update...
Iter 32 with active vertex number 102     
Gauss-seidel update...
Iter 28 with active vertex number 291     
##################### Arguments #####################
depth: 8.
input: /tmp/tmp_783346.obj.
output: /tmp/tmpout_783346.obj.
#####################################################
vertex number: 3911    face number: 1888
Initialize AABB Tree...
Build Halfedges...
Split non-manifold vertices...
Rebuild Halfedges...
Gauss-seidel update...
Iter 24 with active vertex number 149     
Sharp preservation...
Gauss-seidel update...
Iter 18 with active vertex number 44     